In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import time
import copy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map_amsterdam = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map_amsterdam['length']
objective2 = map_amsterdam['crossing']
objective3 = map_amsterdam['obstacle_free_width']

objectives = ('length', 'crossing')

In [6]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map_amsterdam, approach='primal')
nodes = G.nodes
edges = G.edges
print(G)

MultiGraph with 179 nodes and 230 edges


/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
# print(nodes)

In [4]:
#Pick random ones or pick manually that make sense - to experiment
S = (122245.37633330293, 486126.8581684635)
# T = (122246.77932030056, 486223.5791244763)
T = (122253.09793657108, 486219.18429932056)

In [9]:
# TODO: Does it need more time to converge or it just loops and never ends?
v_n = {}  # Value vector
next_n = {}
edge_cost = []
cost = []
pareto_optimal_paths = []
t = np.array([157.33,2])
i=0
for n in G:
    v_n[n] = np.array([0,0])  # Initialisation of nodes

    if n == T:  # We've reached the terminal state
        v_n[n] = np.array([0,0])


converged = False
while not converged:
# while i < 1000:
#     i+=1
    converged = True

    for e in G.edges(data=True):
        n1, n2 = e[0], e[1]  # The nodes

        cost = np.array([e[2]['length'], e[2]['crossing']])
        next_n= np.array([v_n[n1], v_n[n2]])
        result = cost + next_n
        # result = min(cost + next_n, next_n)
        if np.any(next_n != result):
            converged = False

        next_n = result

        if np.all(next_n <= t):
            optimal_action = (n1, n2)
            pareto_optimal_paths.append(optimal_action)

    if converged:  # Check for convergence
        break

# print(next_n)

# i=0
# for n in G:
#     for neigh in G.neighbors(n):
#         while n != T:
#             if next_n[n] + next_n[neigh] == t:
#                 print(i)
#                 pareto_optimal_paths.append((n))
#                 # Move to the next state
#                 n = next_n
#                 pareto_optimal_paths.append((T))

# Filter Pareto-optimal paths from S to T
filtered_paths = []
for path in pareto_optimal_paths:
    if path[0] == S and path[1] == T:
        filtered_paths.append(path)

# Print the Pareto-optimal paths from S to T
for path in filtered_paths:
    print(path)

KeyboardInterrupt: 

In [36]:
def pareto_dominates(a, b):
    """Check if the vector in b Pareto dominates vector a.

    Note: The original code has been modified to work for our minimization problem.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector b dominates vector a.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)


def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs


def pvi(G, T, objectives, max_iter=100, epsilon=0.05):
    # num_states = len(G.nodes)
    start = time.time()
    # cost = []
    nd_vectors = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]
    # nd_vectors = init_pcs

    last_iter = max_iter - 2  # Denotes the start of the last run.
    is_last = False
    save_iteration = False

    # nd_vectors = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]  # Initialisation of nodes

    for n, current_node in enumerate(G.nodes):
        if current_node == T:  # We've reached the terminal state
            # G_copy = G.copy()
            # G_copy.remove_edge(T,n)
            nd_vectors[n] = [{(0, 0)} for _ in G.nodes]
    # print(nd_vectors)

    nd_vectors_update = copy.deepcopy(nd_vectors)
    for run in range(max_iter):  # We execute the algorithm for a number of iterations.
        print(f'Value Iteration number: {run}')
        for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
            if current_node == T:
                continue

            for nk, neighbor in enumerate(G.nodes): #Note: neighbor is an object; k=number
                if neighbor not in G.neighbors(current_node):
                    continue
                # if neighbor == T:
                #     raise
                edge = G[current_node][neighbor]
                edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties

                cost = []
                for i in objectives:
                  cost.append(edge_list[0][i])

                cost = np.array(cost)
                results = set()

                for v_list in nd_vectors[nk]:
                    # if neighbor == T:
                        # print(v_list)
                    for value_vec in v_list:
                        results.add(tuple(cost+value_vec))
                        # if np.all((0, 0) == value_vec):
                            # print(value_vec)
                # for value_vec in nd_vectors[n][nk]:
                #     results.add(tuple(map(sum, zip(cost, value_vec))))
                #     print(results)
                # results = {tuple(cost + value_vec) for v_list in nd_vectors[nk] for value_vec in v_list}  # The set of candidate vectors.
                results = p_prune(results)
                # print(results)
                # print(f"n: {n}; nk:{nk}")
                nd_vectors_update[n][nk] = results

        if is_last:
            break
        nd_vectors = copy.deepcopy(nd_vectors_update)  # Else perform a deep copy and go again.

    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return nd_vectors_update


# for each action in A:
# retireve R=cost
# Q is set of value vector
# s' is neighbor
# T is 1
#actions = neighbors

def track_policy(G, t, objectives, pvi_vec):
    for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
        if current_node == T:
            break

        for nk, neighbor in enumerate(G.neighbors(current_node)):
            edge = G[current_node][neighbor]
            edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties
            cost = []
            for i in objectives:
                cost = np.array(edge_list[0][i])

            for val_vec in pvi_vec:
                if (val_vec + cost) == t:
                    current_node = neighbor
                    t = val_vec


# def track_policy(init_pcs, G, target, tol=1e-3):
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(2)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = np.zeros((n, nk, 2))[n, nk]
#                     non_dominated_set = init_pcs[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew


# init_pcs = [[{(100000, 100000)} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]  # Q-set
# print(init_pcs)
# init_pcs = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]  # Q-set
pvi_result = pvi(G, T, objectives)
print(pvi_result)
t = np.array([165.21, 2.])
path_pvi = track_policy(G, t, objectives, pvi_result)
# print(path_pvi)

Value Iteration number: 0
Value Iteration number: 1
Value Iteration number: 2
Value Iteration number: 3
Value Iteration number: 4
Value Iteration number: 5
Value Iteration number: 6
Value Iteration number: 7
Value Iteration number: 8
Value Iteration number: 9
Value Iteration number: 10
Value Iteration number: 11
Value Iteration number: 12
Value Iteration number: 13
Value Iteration number: 14
Value Iteration number: 15
Value Iteration number: 16
Value Iteration number: 17
Value Iteration number: 18
Value Iteration number: 19
Value Iteration number: 20
Value Iteration number: 21
Value Iteration number: 22
Value Iteration number: 23
Value Iteration number: 24
Value Iteration number: 25
Value Iteration number: 26
Value Iteration number: 27
Value Iteration number: 28
Value Iteration number: 29
Value Iteration number: 30
Value Iteration number: 31
Value Iteration number: 32
Value Iteration number: 33
Value Iteration number: 34
Value Iteration number: 35
Value Iteration number: 36
Value Itera

TypeError: unsupported operand type(s) for +: 'set' and 'int'

In [ ]:
# def track_policy():
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         # state, _ = env.reset()
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(self.num_objectives)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = self.avg_reward[n, nk]
#                     non_dominated_set = self.non_dominated[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew
